In [1]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col

In [2]:
sp = SparkSession.builder.appName("Ranker").getOrCreate()
sp

22/10/06 04:48:51 WARN Utils: Your hostname, J-L resolves to a loopback address: 127.0.1.1; using 172.19.38.60 instead (on interface eth0)
22/10/06 04:48:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/06 04:48:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/06 04:48:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
trans = sp.read.option("inferSchema", True).parquet("../data/curated/transactions/")
trans.show(2)

+-------+------------+------------+--------------+-----------+
|user_id|merchant_abn|dollar_value|order_datetime|   order_id|
+-------+------------+------------+--------------+-----------+
|  14935| 79417999332|      136.07|    2021-11-26|68719476736|
|      1| 46451548968|       72.62|    2021-11-26|68719476737|
+-------+------------+------------+--------------+-----------+
only showing top 2 rows



#### Market share

In [4]:
market_share = trans.groupBy("merchant_abn").count().withColumn("market_share", col("count")/trans.count())
market_share.show(3)

+------------+-----+--------------------+
|merchant_abn|count|        market_share|
+------------+-----+--------------------+
| 83412691377|14288|0.001006515794964...|
| 38700038932| 7132|5.024125594686487E-4|
| 35344855546| 1522|1.072170380694452...|
+------------+-----+--------------------+
only showing top 3 rows



#### Average dollar sum value for each merchant /number of customers that merchant has

In [5]:
number_of_transactions_by_merchant = trans.groupBy("merchant_abn").count()

In [ ]:

number_of_total_dollar_value = trans.groupBy("merchant_abn").sum("dollar_value")
number_of_total_dollar_value = number_of_total_dollar_value.withColumnRenamed("merchant_abn","merchant_id")
number_of_average_dollar_value = number_of_transactions_by_merchant.join(number_of_total_dollar_value,number_of_total_dollar_value.merchant_id == number_of_users_by_merchant.merchant_abn,"inner")
number_of_average_dollar_value = number_of_average_dollar_value.withColumn("Average dollar value per number of customers",col("sum(dollar_value)")/col("count"))
number_of_average_dollar_value = number_of_average_dollar_value.drop(number_of_average_dollar_value.merchant_id)
number_of_average_dollar_value = number_of_average_dollar_value.drop(number_of_average_dollar_value['count'])
number_of_average_dollar_value = number_of_average_dollar_value.drop(number_of_average_dollar_value['sum(dollar_value)'])
number_of_average_dollar_value.show(5)